## Load Data from CSVs

In [74]:
import unicodecsv

## Longer version of code (replaced with shorter, equivalent version below)

# enrollments = []
# f = open('enrollments.csv', 'rb')
# reader = unicodecsv.DictReader(f)
# for row in reader:
#     enrollments.append(row)
# f.close()

with open('enrollments.csv', 'rb') as f:
    reader = unicodecsv.DictReader(f)
    enrollments = list(reader)

In [4]:
#####################################
#                 1                 #
#####################################

## Read in the data from daily_engagement.csv and project_submissions.csv 
## and store the results in the below variables.
## Then look at the first row of each table.
with open('daily_engagement.csv', 'rb') as f:
    reader = unicodecsv.DictReader(f)
    daily_engagement = list(reader)

with open('project_submissions.csv', 'rb') as f:
    reader = unicodecsv.DictReader(f)
    project_submissions = list(reader)    


## Fixing Data Types

In [75]:
from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
enrollments[0]

{'account_key': '448',
 'status': 'canceled',
 'join_date': datetime.datetime(2014, 11, 10, 0, 0),
 'cancel_date': datetime.datetime(2015, 1, 14, 0, 0),
 'days_to_cancel': 65,
 'is_udacity': True,
 'is_canceled': True}

In [6]:
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    
daily_engagement[0]

{'account_key': '0',
 'utc_date': datetime.datetime(2015, 1, 9, 0, 0),
 'num_courses_visited': 1,
 'total_minutes_visited': 11.6793745,
 'lessons_completed': 0,
 'projects_completed': 0}

In [7]:
# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])

project_submissions[0]

{'creation_date': datetime.datetime(2015, 1, 14, 0, 0),
 'completion_date': datetime.datetime(2015, 1, 16, 0, 0),
 'assigned_rating': 'UNGRADED',
 'account_key': '256',
 'lesson_key': '3176718735',
 'processing_state': 'EVALUATED'}

## Investigating the Data

In [76]:
import pandas as pd
enrollments_df = pd.DataFrame(enrollments)
daily_engagement_df = pd.DataFrame(daily_engagement)
project_submissions_df = pd.DataFrame(project_submissions)


In [16]:
enrollments_df.shape

(1640, 7)

In [22]:
enrollments_df.account_key.nunique()

1302

In [17]:
daily_engagement_df.shape

(136240, 6)

In [31]:
daily_engagement_df.account_key.unique()

array(['0', '1', '2', ..., '1302', '1303', '1305'], dtype=object)

In [18]:
project_submissions_df.shape

(3642, 6)

In [24]:
project_submissions_df.account_key.nunique()

743

In [28]:
daily_engagement_df

,account_key,utc_date,num_courses_visited,total_minutes_visited,lessons_completed,projects_completed
0,0,2015-01-09,1,11.679374,0,0
1,0,2015-01-10,2,37.284887,0,0
2,0,2015-01-11,2,53.633746,0,0
3,0,2015-01-12,1,33.489270,0,0
4,0,2015-01-13,1,64.779678,0,0
...,...,...,...,...,...,...
136235,1305,2015-06-14,0,0.000000,0,0
136236,1305,2015-06-15,0,0.000000,0,0
136237,1305,2015-06-16,1,2.720136,0,0
136238,1305,2015-06-17,0,0.000000,0,0


## Problems in the Data

In [36]:
daily_uniq = daily_engagement_df.account_key.unique()

In [45]:
missing_stud = []
for df in enrollments_df.account_key:
    if df not in daily_uniq:
        missing_stud.append(df)
len(missing_stud)

71

In [55]:
missing_stud_df = enrollments_df.loc[enrollments_df.account_key.isin(missing_stud)]
missing_stud_df

,account_key,status,join_date,cancel_date,days_to_cancel,is_udacity,is_canceled
65,1219,canceled,2014-11-12,2014-11-12,0.0,False,True
75,871,canceled,2014-11-13,2014-11-13,0.0,False,True
193,1218,canceled,2014-11-15,2014-11-15,0.0,False,True
216,654,canceled,2014-12-04,2014-12-04,0.0,False,True
217,654,canceled,2014-12-04,2014-12-04,0.0,False,True
...,...,...,...,...,...,...,...
1501,1079,canceled,2015-02-09,2015-02-09,0.0,False,True
1543,1063,canceled,2015-07-14,2015-07-14,0.0,False,True
1563,1270,canceled,2015-07-20,2015-07-20,0.0,False,True
1608,1291,canceled,2015-06-03,2015-06-03,0.0,False,True


In [54]:
missing_stud_df.query('days_to_cancel != 0')

,account_key,status,join_date,cancel_date,days_to_cancel,is_udacity,is_canceled
467,1304,canceled,2015-01-10,2015-03-10,59.0,True,True
468,1304,canceled,2015-03-10,2015-06-17,99.0,True,True
1378,1101,current,2015-02-25,NaT,NaN,True,False


## Missing Engagement Records

In [ ]:
#####################################
#                 4                 #
#####################################

## Find any one student enrollments where the student is missing from the daily engagement table.
## Output that enrollment.

## Checking for More Problem Records

In [ ]:
#####################################
#                 5                 #
#####################################

## Find the number of surprising data points (enrollments missing from
## the engagement table) that remain, if any.

## Tracking Down the Remaining Problems

In [88]:
def remove_udacity_accounts(data):
    l = []
    for i in data:
        if i['account_key'] not in ['448', '1304', '312', '818', '1069', '1101']:
            l.append(i)
    return l

In [95]:
# Remove Udacity test accounts from all three tables
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

#ТУТ ФРЕЙМЫ
enrollments_df = pd.DataFrame(non_udacity_enrollments)
daily_engagement_df = pd.DataFrame(non_udacity_engagement)
project_submissions_df = pd.DataFrame(non_udacity_submissions)

## Refining the Question

In [105]:

## Create a dictionary named paid_students containing all students who either
## haven't canceled yet or who remained enrolled for more than 7 days. The keys
## should be account keys, and the values should be the date the student enrolled.

paid_students = enrollments_df.query('(days_to_cancel > 7) or (days_to_cancel == "Nan")')


In [165]:
d = paid_students.account_key.unique()

In [167]:
daily_engagement_df = daily_engagement_df.loc[daily_engagement_df.account_key.isin(d)]
daily_engagement_df

,account_key,utc_date,num_courses_visited,total_minutes_visited,lessons_completed,projects_completed
0,0,2015-01-09,1,11.679374,0,0
1,0,2015-01-10,2,37.284887,0,0
2,0,2015-01-11,2,53.633746,0,0
3,0,2015-01-12,1,33.489270,0,0
4,0,2015-01-13,1,64.779678,0,0
...,...,...,...,...,...,...
135651,1305,2015-06-14,0,0.000000,0,0
135652,1305,2015-06-15,0,0.000000,0,0
135653,1305,2015-06-16,1,2.720136,0,0
135654,1305,2015-06-17,0,0.000000,0,0


In [ ]:
135656

## Getting Data from First Week

In [ ]:
# Takes a student's join date and the date of a specific engagement record,
# and returns True if that engagement record happened within one week
# of the student joining.
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days < 7

In [214]:
paid_students_date  = paid_students[['account_key','join_date']]
paid_students_date = paid_students_date.groupby('account_key', as_index=False).agg({'join_date':'max'})
paid_students_date

,account_key,join_date
0,0,2015-01-09
1,1,2015-05-06
2,10,2014-11-10
3,100,2015-02-05
4,1000,2015-07-11
...,...,...
990,988,2015-05-13
991,989,2015-04-08
992,99,2015-03-10
993,990,2015-08-23


In [220]:
paid_engagement = daily_engagement_df.merge(paid_students_date, on='account_key', how='left')
paid_engagement['time_delta'] = (paid_engagement.utc_date\
                                               - paid_engagement.join_date)\
                                                .dt.days

## Create a list of rows from the engagement table including only rows where
## the student is one of the paid students you just found, and the date is within
## one week of the student's join date.




In [223]:
paid_engagement_in_first_week = paid_engagement.query('time_delta < 7')
paid_engagement_in_first_week.shape

# 21508 записей или активностей за неделю 

(21508, 8)

## Exploring Student Engagement

In [ ]:
from collections import defaultdict

# Create a dictionary of engagement grouped by student.
# The keys are account keys, and the values are lists of engagement records.
engagement_by_account = defaultdict(list)
for engagement_record in paid_engagement_in_first_week:
    account_key = engagement_record['account_key']
    engagement_by_account[account_key].append(engagement_record)

In [ ]:
# Create a dictionary with the total minutes each student spent in the classroom during the first week.
# The keys are account keys, and the values are numbers (total minutes)
total_minutes_by_account = {}
for account_key, engagement_for_student in engagement_by_account.items():
    total_minutes = 0
    for engagement_record in engagement_for_student:
        total_minutes += engagement_record['total_minutes_visited']
    total_minutes_by_account[account_key] = total_minutes

In [ ]:
import numpy as np

# Summarize the data about minutes spent in the classroom
total_minutes = total_minutes_by_account.values()
print 'Mean:', np.mean(total_minutes)
print 'Standard deviation:', np.std(total_minutes)
print 'Minimum:', np.min(total_minutes)
print 'Maximum:', np.max(total_minutes)

In [224]:
paid_engagement_in_first_week.head()

,account_key,utc_date,num_courses_visited,total_minutes_visited,lessons_completed,projects_completed,join_date,time_delta
0,0,2015-01-09,1,11.679374,0,0,2015-01-09,0
1,0,2015-01-10,2,37.284887,0,0,2015-01-09,1
2,0,2015-01-11,2,53.633746,0,0,2015-01-09,2
3,0,2015-01-12,1,33.489270,0,0,2015-01-09,3
4,0,2015-01-13,1,64.779678,0,0,2015-01-09,4


In [247]:
paid_engagement_in_first_week.query('account_key == "99"')

,account_key,utc_date,num_courses_visited,total_minutes_visited,lessons_completed,projects_completed,join_date,time_delta
16657,99,2015-01-13,1,21.231360,0,0,2015-03-10,-56
16658,99,2015-01-14,0,0.000000,0,0,2015-03-10,-55
16659,99,2015-01-15,2,31.008425,0,0,2015-03-10,-54
16660,99,2015-01-16,0,0.000000,0,0,2015-03-10,-53
16661,99,2015-01-17,0,0.000000,0,0,2015-03-10,-52
...,...,...,...,...,...,...,...,...
16715,99,2015-03-12,0,0.000000,0,0,2015-03-10,2
16716,99,2015-03-13,1,113.697930,0,0,2015-03-10,3
16717,99,2015-03-14,0,0.000000,0,0,2015-03-10,4
16718,99,2015-03-15,0,0.000000,0,0,2015-03-10,5


In [283]:
paid_engagement_in_first_week_new = paid_engagement_in_first_week.query('utc_date  >= join_date')
paid_engagement_in_first_week_new

,account_key,utc_date,num_courses_visited,total_minutes_visited,lessons_completed,projects_completed,join_date,time_delta
0,0,2015-01-09,1,11.679374,0,0,2015-01-09,0
1,0,2015-01-10,2,37.284887,0,0,2015-01-09,1
2,0,2015-01-11,2,53.633746,0,0,2015-01-09,2
3,0,2015-01-12,1,33.489270,0,0,2015-01-09,3
4,0,2015-01-13,1,64.779678,0,0,2015-01-09,4
...,...,...,...,...,...,...,...,...
134513,1305,2015-05-14,1,7.715250,0,0,2015-05-12,2
134514,1305,2015-05-15,0,0.000000,0,0,2015-05-12,3
134515,1305,2015-05-16,0,0.000000,0,0,2015-05-12,4
134516,1305,2015-05-17,0,0.000000,0,0,2015-05-12,5


In [252]:
total_minutes_by_account = paid_engagement_in_first_week_new\
                            .groupby('account_key', as_index=False)\
                            .agg({'total_minutes_visited':'sum'})


In [254]:
total_minutes_by_account.describe()

,total_minutes_visited
count,995.000000
mean,306.708327
std,413.204626
min,0.000000
25%,13.743180
50%,129.859204
75%,451.903246
max,3564.733264


## Debugging Data Analysis Code

In [ ]:
#####################################
#                 8                 #
#####################################

## Go through a similar process as before to see if there is a problem.
## Locate at least one surprising piece of data, output it, and take a look at it.

## Lessons Completed in First Week

In [255]:
total_lessons_by_account = paid_engagement_in_first_week_new\
                            .groupby('account_key', as_index=False)\
                            .agg({'lessons_completed':'sum'})
total_lessons_by_account.describe()

## Adapt the code above to find the mean, standard deviation, minimum, and maximum for
## the number of lessons completed by each student during the first week. Try creating
## one or more functions to re-use the code above.

,lessons_completed
count,995.000000
mean,1.636181
std,3.004071
min,0.000000
25%,0.000000
50%,0.000000
75%,2.000000
max,36.000000


## Number of Visits in First Week

In [274]:
has_visited = [1 if i > 0  else 0 for i in paid_engagement_in_first_week_new.num_courses_visited ] 

## Find the mean, standard deviation, minimum, and maximum for the number of
## days each student visits the classroom during the first week.

In [289]:
paid_engagement_in_first_week_new['has_visited'] = has_visited

<ipython-input-289-a246d72b90ad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paid_engagement_in_first_week_new['has_visited'] = has_visited


In [292]:
number_of_visits = paid_engagement_in_first_week_new\
                            .groupby('account_key', as_index=False)\
                            .agg({'has_visited':'sum'})


In [293]:
number_of_visits.describe()

,has_visited
count,995.000000
mean,2.867337
std,2.256332
min,0.000000
25%,1.000000
50%,3.000000
75%,5.000000
max,7.000000


## Splitting out Passing Students

In [ ]:
######################################
#                 11                 #
######################################

## Create two lists of engagement data for paid students in the first week.
## The first list should contain data for students who eventually pass the
## subway project, and the second list should contain data for students
## who do not.

subway_project_lesson_keys = ['746169184', '3176718735']

passing_engagement =
non_passing_engagement =

## Comparing the Two Student Groups

In [ ]:
######################################
#                 12                 #
######################################

## Compute some metrics you're interested in and see how they differ for
## students who pass the subway project vs. students who don't. A good
## starting point would be the metrics we looked at earlier (minutes spent
## in the classroom, lessons completed, and days visited).

## Making Histograms

In [ ]:
######################################
#                 13                 #
######################################

## Make histograms of the three metrics we looked at earlier for both
## students who passed the subway project and students who didn't. You
## might also want to make histograms of any other metrics you examined.

## Improving Plots and Sharing Findings

In [ ]:
######################################
#                 14                 #
######################################

## Make a more polished version of at least one of your visualizations
## from earlier. Try importing the seaborn library to make the visualization
## look better, adding axis labels and a title, and changing one or more
## arguments to the hist() function.